# Homework

In [ ]:
library(tidyverse)
df <- read.csv("flights.csv")
df %>% glimpse()

Rows: 336,776
Columns: 19
$ year           <int> 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2…
$ month          <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ day            <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ dep_time       <int> 517, 533, 542, 544, 554, 554, 555, 557, 557, 558, 558, …
$ sched_dep_time <int> 515, 529, 540, 545, 600, 558, 600, 600, 600, 600, 600, …
$ dep_delay      <int> 2, 4, 2, -1, -6, -4, -5, -3, -3, -2, -2, -2, -2, -2, -1…
$ arr_time       <int> 830, 850, 923, 1004, 812, 740, 913, 709, 838, 753, 849,…
$ sched_arr_time <int> 819, 830, 850, 1022, 837, 728, 854, 723, 846, 745, 851,…
$ arr_delay      <int> 11, 20, 33, -18, -25, 12, 19, -14, -8, 8, -2, -3, 7, -1…
$ carrier        <chr> "UA", "UA", "AA", "B6", "DL", "UA", "B6", "EV", "B6", "…
$ flight         <int> 1545, 1714, 1141, 725, 461, 1696, 507, 5708, 79, 301, 4…
$ tailnum        <chr> "N14228", "N24211", "N619AA", "N804JB", "N668DN", "N394…
$ origin      

## Cleaning data

In [ ]:
 sum(complete.cases(df))/ nrow(df)
 # around 3% of rows has missing values, so let's drop them
df <- drop_na(df) %>% tibble()

0.971999192341497


## Q1. Top 5 the most popular destinations

In [ ]:
df %>%
    count(dest) %>%
    arrange(desc(n)) %>%
    head(5)

dest,n
<chr>,<int>
ATL,16837
ORD,16566
LAX,16026
BOS,15022
MCO,13967


## Q2. Which months people traveled a lot?

In [ ]:
df %>% 
    count(month) %>%
    arrange(desc(n)) %>%
    head(3)

month,n
<int>,<int>
8,28756
10,28618
7,28293


## Q3. Where did people go in busy months (7,8,10)

In [ ]:
df %>%
    filter(month %in% c(7,8,10)) %>%
    group_by(month, dest) %>%
    summarise(n = n()) %>% 
    arrange(desc(n)) %>%
    head(9) %>% arrange(month)

`summarise()` has grouped output by 'month'. You can override using the `.groups` argument.



month,dest,n
<int>,<chr>,<int>
7,LAX,1494
7,ORD,1487
7,ATL,1448
8,ORD,1569
8,LAX,1500
8,ATL,1488
10,ORD,1581
10,ATL,1439
10,LAX,1405


## Q4. Which airline delayed the most (arrival)

In [ ]:
# Create new column is_late
df %>%
    mutate(is_delay = case_when(
        arr_delay > 0 ~ T,
        TRUE ~ F
    )) %>%
    select(carrier, is_delay) %>% 
    group_by(carrier) %>%
    summarise(total_delay = sum(is_delay), total_flight = n()) %>%
    mutate(delay_rate = (total_delay/total_flight)) %>%
    arrange(desc(delay_rate)) %>%
    select(c(1,4)) %>% head(5)

carrier,delay_rate
<chr>,<dbl>
FL,0.5968504
F9,0.5756241
EV,0.4790639
YV,0.4742647
MQ,0.4670288


## Q5. How much proportion of top destinations (ATL, ORD, LAX) in total flights of each airline

In [ ]:
top_dest <- c("ATL", "ORD", "LAX")
df %>%
    mutate(is_top_dest = ifelse(dest %in% top_dest, T, F)) %>%
    group_by(carrier) %>%
    summarise(top_dest_prop = mean(is_top_dest),
              n = n(),
              top_dest = sum(is_top_dest)) %>%
    arrange(desc(top_dest_prop))

carrier,top_dest_prop,n,top_dest
<chr>,<dbl>,<int>,<int>
FL,0.717480315,3175,2278
VX,0.499218139,5116,2554
AA,0.293986916,31947,9392
DL,0.271496916,47658,12939
UA,0.218337891,57782,12616
MQ,0.173023925,25037,4332
9E,0.060136464,17294,1040
B6,0.047382930,54049,2561
OO,0.034482759,29,1


# Database connection (PostgreSQL)

### Create 3 Dataframes

In [ ]:
students <- data.frame(
    std_id <- 1:3,
    firstname = c("Somchai", "Somchit", "Somnamnha"),
    lastname = c("Mhaihua", "Ridsiduang", "Ibaharoi"),
    age = c(50, 20, 12)
)

bridge <- data.frame(
    std_id = c(1, 1, 1, 2, 2, 3),
    skills_id = c(1, 2, 3, 3, 1, 3)
)

skills <- data.frame(
    skills_id = 1:3,
    skills = c("Python", "R", "SQL")
)

### Connect to the database

In [ ]:
library(RPostgreSQL)

ERROR: ERROR: ERROR: Error in library(RPostgreSQL): there is no package called ‘RPostgreSQL’
